In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# Chuẩn bị dữ liệu

In [6]:
def read_data(file_path, list_stocks=None, start_time="2010-01-01", end_time="2025-01-01"):
    df = pd.read_csv(file_path)

    if not list_stocks:
        list_stocks = df['stock_code'].unique()

    result_df = {}

    for stock in list_stocks:
        data = df[df['stock_code'] == stock].copy()
        data['transaction_date'] = pd.to_datetime(data['transaction_date'])

        # Lag Features
        data['lag_1'] = data['closing_price'].shift(1)
        data['lag_5'] = data['closing_price'].shift(5)
        data['lag_10'] = data['closing_price'].shift(10)

        # Rolling Mean Features
        data['rolling_mean_5'] = data['closing_price'].rolling(window=5).mean()
        data['rolling_mean_10'] = data['closing_price'].rolling(window=10).mean()
        data['rolling_mean_20'] = data['closing_price'].rolling(window=20).mean()

        # Rolling Std Features
        data['rolling_std_5'] = data['closing_price'].rolling(window=5).std()
        data['rolling_std_10'] = data['closing_price'].rolling(window=10).std()
        data['rolling_std_20'] = data['closing_price'].rolling(window=20).std()

        # Expanding Mean Feature
        data['expanding_mean'] = data['closing_price'].expanding().mean()

        data = data.fillna(0)
        data = data.replace([np.inf, -np.inf], 0)
        data = data[(data["transaction_date"] >= start_time) & (data["transaction_date"] < end_time)]

        feature_columns = ['transaction_date', 'closing_price', 'lag_1', 'lag_5', 'lag_10', 
                           'rolling_mean_5', 'rolling_mean_10', 'rolling_mean_20', 
                           'rolling_std_5', 'rolling_std_10', 'rolling_std_20', 'expanding_mean']

        selected_data = data[feature_columns]
        result_df[stock] = selected_data

    return result_df



In [7]:
def compute_features(history_prices):
   
    import pandas as pd
    import numpy as np
    
    series = pd.Series(history_prices)

    # Giá đóng cửa của ngày mới nhất
    cp = series.iloc[-1]

    # Lag Features
    lag_1 = series.shift(1).iloc[-1] if len(series) >= 2 else 0
    lag_5 = series.shift(5).iloc[-1] if len(series) >= 6 else 0
    lag_10 = series.shift(10).iloc[-1] if len(series) >= 11 else 0

    # Rolling Mean Features
    if len(series) >= 5:
        rolling_mean_5 = series.rolling(window=5).mean().iloc[-1]
    else:
        rolling_mean_5 = series.mean()
        
    if len(series) >= 10:
        rolling_mean_10 = series.rolling(window=10).mean().iloc[-1]
    else:
        rolling_mean_10 = series.mean()
        
    if len(series) >= 20:
        rolling_mean_20 = series.rolling(window=20).mean().iloc[-1]
    else:
        rolling_mean_20 = series.mean()

    # Rolling Std Features
    if len(series) >= 5:
        rolling_std_5 = series.rolling(window=5).std().iloc[-1]
    else:
        rolling_std_5 = series.std()
        
    if len(series) >= 10:
        rolling_std_10 = series.rolling(window=10).std().iloc[-1]
    else:
        rolling_std_10 = series.std()
        
    if len(series) >= 20:
        rolling_std_20 = series.rolling(window=20).std().iloc[-1]
    else:
        rolling_std_20 = series.std()

    # Expanding Mean Feature
    expanding_mean = series.expanding().mean().iloc[-1]

    return [cp, lag_1, lag_5, lag_10,
            rolling_mean_5, rolling_mean_10, rolling_mean_20,
            rolling_std_5, rolling_std_10, rolling_std_20,
            expanding_mean]


In [ ]:
def prepare_data(stock_data, sequence_length=10, target_column='closing_price', split_date = "2024-01-01"):
    """
    Chuẩn bị dữ liệu cho mô hình LSTM và XGBoost, bao gồm chuẩn hóa cho cả hai mô hình
    
    Parameters:
    -----------
    stock_data : pandas DataFrame
        Dữ liệu cổ phiếu đã được xử lý
    sequence_length : int
        Độ dài chuỗi thời gian cho LSTM
    target_column : str
        Tên cột dữ liệu mục tiêu cần dự đoán
    test_size : float
        Tỷ lệ dữ liệu dùng cho tập test
        
    Returns:
    --------
    Dictionary chứa dữ liệu đã xử lý cho LSTM và XGBoost
    """
    # Sắp xếp dữ liệu theo thời gian
    stock_data = stock_data.sort_values('transaction_date')
    df_train = stock_data[stock_data['transaction_date'] < split_date]
    df_test = stock_data[stock_data['transaction_date'] >= split_date]
    # Chuẩn bị dữ liệu
    feature_columns = [col for col in stock_data.columns if col not in ['transaction_date', target_column]]
    
    
    X_train= df_train[feature_columns].values
    X_test = df_test[feature_columns].values
    y_train = df_train[target_column].values
    y_test =  df_test[target_column].values
    

    scaler_X = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)

    y_train_reshaped = y_train.reshape(-1, 1)
    y_test_reshaped = y_test.reshape(-1, 1)
    
    y_train_scaled = scaler_y.fit_transform(y_train_reshaped)
    y_test_scaled = scaler_y.transform(y_test_reshaped)
    

    X_train_xgb = X_train_scaled
    X_test_xgb = X_test_scaled
    y_train_xgb = y_train_scaled.flatten()
    y_test_xgb = y_test_scaled.flatten()

    X_sequences, y_sequences = [], []
    for i in range(len(X_train_scaled) - sequence_length):
        X_sequences.append(X_train_scaled[i:i+sequence_length])
        y_sequences.append(y_train_scaled[i+sequence_length])
    
    X_sequences = np.array(X_sequences)
    y_sequences = np.array(y_sequences)
    

    X_test_sequences, y_test_sequences = [], []
    for i in range(len(X_test_scaled) - sequence_length):
        X_test_sequences.append(X_test_scaled[i:i+sequence_length])
        y_test_sequences.append(y_test_scaled[i+sequence_length])

    if len(X_test_scaled) < sequence_length:
        combined_data = np.vstack((X_train_scaled[-sequence_length+len(X_test_scaled):], X_test_scaled))
        X_test_sequences = [combined_data[:sequence_length]]
        y_test_sequences = [y_test_scaled[0]]
    
    X_test_sequences = np.array(X_test_sequences)
    y_test_sequences = np.array(y_test_sequences)
    
    return {
        'xgboost': {
            'X_train': X_train_xgb, 
            'y_train': y_train_xgb,
            'X_test': X_test_xgb, 
            'y_test': y_test_xgb,
            'feature_names': feature_columns,
            'scaler_X': scaler_X,
            'scaler_y': scaler_y
        },
        'lstm': {
            'X_train': X_sequences, 
            'y_train': y_sequences,
            'X_test': X_test_sequences, 
            'y_test': y_test_sequences,
            'scaler_y': scaler_y,
            'scaler_X': scaler_X,
            'sequence_length': sequence_length
        },
        'dates_test': stock_data['transaction_date'].values[-len(y_test):],
        'actual_test': y_test
    }

# Mô hình LSTM

In [40]:
def build_lstm_model(prepared_data, units=50, dropout_rate=0.2, learning_rate=0.001, epochs=100, batch_size=32, patience=10):
    """
    Xây dựng và huấn luyện mô hình LSTM
    
    Parameters:
    -----------
    prepared_data : dict
        Dictionary chứa dữ liệu đã chuẩn bị cho LSTM
    units : int
        Số lượng units trong layer LSTM
    dropout_rate : float
        Tỷ lệ dropout để tránh overfitting
    learning_rate : float
        Tốc độ học của optimizer
    epochs : int
        Số lượng epochs huấn luyện
    batch_size : int
        Kích thước batch
    patience : int
        Số epochs đợi trước khi early stopping
        
    Returns:
    --------
    Mô hình LSTM đã huấn luyện và kết quả dự đoán
    """

    X_train = prepared_data['lstm']['X_train']
    y_train = prepared_data['lstm']['y_train']
    X_test = prepared_data['lstm']['X_test']
    scaler_y = prepared_data['lstm']['scaler_y']
    

    input_shape = (X_train.shape[1], X_train.shape[2])
    
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        callbacks=[early_stopping],
        verbose=1
    )

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    
    return {
        'model': model,
        'history': history,
        'predictions': y_pred.flatten(),
        'predictions_scaled': y_pred_scaled.flatten()
    }

# Mô hình XGBoost

In [41]:
def build_xgboost_model(prepared_data, max_depth=7, learning_rate=0.1, n_estimators=100, early_stopping_rounds=10):
    """
    Xây dựng và huấn luyện mô hình XGBoost với dữ liệu đã chuẩn hóa
    
    Parameters:
    -----------
    prepared_data : dict
        Dictionary chứa dữ liệu đã chuẩn bị cho XGBoost
    max_depth : int
        Độ sâu tối đa của cây
    learning_rate : float
        Tốc độ học của mô hình
    n_estimators : int
        Số lượng cây ước lượng
    early_stopping_rounds : int
        Số vòng đợi trước khi early stopping
        
    Returns:
    --------
    Mô hình XGBoost đã huấn luyện và kết quả dự đoán
    """
    # Lấy dữ liệu
    X_train = prepared_data['xgboost']['X_train']
    y_train = prepared_data['xgboost']['y_train']
    X_test = prepared_data['xgboost']['X_test']
    feature_names = prepared_data['xgboost']['feature_names']
    scaler_y = prepared_data['xgboost']['scaler_y']
    
    # Tạo tập validation từ tập training
    X_train_xgb, X_val_xgb, y_train_xgb, y_val_xgb = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    
    # Khởi tạo mô hình XGBoost
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        random_state=42,
        n_jobs=-1
    )

    model.fit(
        X_train_xgb, y_train_xgb,
        eval_set=[(X_val_xgb, y_val_xgb)],
        verbose=True
    )
    
    # Dự đoán trên tập test (vẫn ở dạng đã chuẩn hóa)
    y_pred_scaled = model.predict(X_test)
    
    # Chuyển đổi về giá trị thực
    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    
    # Lấy feature importance
    feature_importance = model.feature_importances_
    feature_importance_dict = {feature: importance for feature, importance in zip(feature_names, feature_importance)}
    
    return {
        'model': model,
        'predictions': y_pred,
        'predictions_scaled': y_pred_scaled,
        'feature_importance': feature_importance_dict
    }


# Trực quan hóa kết quả
def visualize_predictions(dates, actual, lstm_pred, xgb_pred, stock_code):
    """
    Trực quan hóa kết quả dự đoán
    
    Parameters:
    -----------
    dates : array
        Các ngày trong tập test
    actual : array
        Giá trị thực tế
    lstm_pred : array
        Dự đoán từ mô hình LSTM
    xgb_pred : array
        Dự đoán từ mô hình XGBoost
    stock_code : str
        Mã cổ phiếu
    """
    plt.figure(figsize=(14, 7))
    plt.plot(dates, actual, label='Giá thực tế', color='black', linewidth=2)
    plt.plot(dates, lstm_pred, label='Dự đoán LSTM', color='blue', linestyle='--')
    plt.plot(dates, xgb_pred, label='Dự đoán XGBoost', color='red', linestyle='-.')
    
    plt.title(f'So sánh dự đoán giá cổ phiếu {stock_code}', fontsize=16)
    plt.xlabel('Ngày', fontsize=12)
    plt.ylabel('Giá đóng cửa', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Biểu đồ độ lỗi
    plt.figure(figsize=(14, 7))
    plt.subplot(2, 1, 1)
    plt.plot(dates, actual - lstm_pred, label='Sai số LSTM', color='blue')
    plt.title(f'Sai số dự đoán - LSTM', fontsize=14)
    plt.xlabel('Ngày', fontsize=12)
    plt.ylabel('Sai số', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(2, 1, 2)
    plt.plot(dates, actual - xgb_pred, label='Sai số XGBoost', color='red')
    plt.title(f'Sai số dự đoán - XGBoost', fontsize=14)
    plt.xlabel('Ngày', fontsize=12)
    plt.ylabel('Sai số', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

    # Thêm biểu đồ so sánh phần trăm sai số
    pct_error_lstm = np.abs((actual - lstm_pred) / actual) * 100
    pct_error_xgb = np.abs((actual - xgb_pred) / actual) * 100
    
    plt.figure(figsize=(14, 7))
    plt.plot(dates, pct_error_lstm, label='% Sai số LSTM', color='blue')
    plt.plot(dates, pct_error_xgb, label='% Sai số XGBoost', color='red')
    plt.title(f'Phần trăm sai số dự đoán (%)', fontsize=14)
    plt.xlabel('Ngày', fontsize=12)
    plt.ylabel('Phần trăm sai số (%)', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [42]:
def predict_next_day(latest_data, lstm_model, xgb_model, prepared_data):
    """
    Dự đoán giá cổ phiếu cho ngày tiếp theo
    
    Parameters:
    -----------
    latest_data : array
        Dữ liệu mới nhất cho dự đoán
    lstm_model : Keras model
        Mô hình LSTM đã huấn luyện
    xgb_model : XGBoost model
        Mô hình XGBoost đã huấn luyện
    prepared_data : dict
        Dictionary chứa dữ liệu đã chuẩn bị
        
    Returns:
    --------
    Dictionary chứa dự đoán từ cả hai mô hình
    """
    # Lấy các scalers và thông số
    scaler_X = prepared_data['lstm']['scaler_X']
    scaler_y = prepared_data['lstm']['scaler_y']
    sequence_length = prepared_data['lstm']['sequence_length']
    
    # Chuẩn bị dữ liệu đầu vào - chuẩn hóa dữ liệu mới
    latest_scaled = scaler_X.transform(latest_data.reshape(1, -1))
    
    # Nếu có đủ dữ liệu trong chuỗi cho LSTM
    if len(latest_scaled) >= sequence_length:
        latest_sequence = latest_scaled[-sequence_length:].reshape(1, sequence_length, latest_scaled.shape[1])
        lstm_pred_scaled = lstm_model.predict(latest_sequence)
        lstm_pred = scaler_y.inverse_transform(lstm_pred_scaled)[0][0]
    else:
        lstm_pred = None
    
    # Dự đoán với XGBoost sử dụng dữ liệu đã chuẩn hóa
    xgb_pred_scaled = xgb_model.predict(latest_scaled)[0]
    xgb_pred = scaler_y.inverse_transform(np.array([[xgb_pred_scaled]]))[0][0]
    
    return {
        'lstm_prediction': lstm_pred,
        'xgboost_prediction': xgb_pred
    }

In [66]:
# Đánh giá mô hình
def evaluate_models(actual, lstm_pred, xgb_pred):
    results = {}
    all_predicts = {}
    # Đánh giá LSTM
    results['lstm'] = {
        'rmse': np.sqrt(mean_squared_error(actual, lstm_pred)),
        'mae': mean_absolute_error(actual, lstm_pred),
        'r2': r2_score(actual, lstm_pred),
        'mse': mean_squared_error(actual, lstm_pred)
    }

    all_predicts['lstm'] = {'actual': actual,
        'predictions': lstm_pred}
    
    # Đánh giá XGBoost
    results['xgboost'] = {
        'rmse': np.sqrt(mean_squared_error(actual, xgb_pred)),
        'mae': mean_absolute_error(actual, xgb_pred),
        'r2': r2_score(actual, xgb_pred),
        'mse': mean_squared_error(actual, xgb_pred)
    }

    all_predicts['xgboost'] = {'actual': actual,
                                'predictions': xgb_pred}
    
    # So sánh hiệu suất
    print("Đánh giá mô hình LSTM:")
    print(f"RMSE: {results['lstm']['rmse']:.4f}")
    print(f"MAE: {results['lstm']['mae']:.4f}")
    print(f"R²: {results['lstm']['r2']:.4f}")
    print("\nĐánh giá mô hình XGBoost:")
    print(f"RMSE: {results['xgboost']['rmse']:.4f}")
    print(f"MAE: {results['xgboost']['mae']:.4f}")
    print(f"R²: {results['xgboost']['r2']:.4f}")
    
    return results, all_predicts


In [70]:
# Hàm chính để chạy toàn bộ quá trình
def run_stock_prediction(file_path, stock_code):
    """
    Chạy toàn bộ quy trình dự đoán giá cổ phiếu
    
    Parameters:
    -----------
    file_path : str
        Đường dẫn đến file dữ liệu
    stock_code : str
        Mã cổ phiếu cần dự đoán
    """
    print(f"Bắt đầu dự đoán giá cổ phiếu {stock_code}...")
    squence_length = 20
    # Đọc dữ liệu
    stocks_data = read_data(file_path, [stock_code])
    
    if not stocks_data or stock_code not in stocks_data:
        print(f"Không tìm thấy dữ liệu cho cổ phiếu {stock_code}")
        return
    
    stock_data = stocks_data[stock_code]
    print(f"Đã đọc dữ liệu cho cổ phiếu {stock_code}, số lượng mẫu: {len(stock_data)}")
    
    # Chuẩn bị dữ liệu
    prepared_data = prepare_data(stock_data, sequence_length=squence_length)
    print("Hoàn thành chuẩn bị dữ liệu")
    
    # Huấn luyện mô hình LSTM
    print("Đang huấn luyện mô hình LSTM...")
    lstm_results = build_lstm_model(
        prepared_data, 
        units=64, 
        dropout_rate=0.2, 
        learning_rate=0.001, 
        epochs=20, 
        batch_size=16
    )
    print("Hoàn thành huấn luyện mô hình LSTM")
    
    # Huấn luyện mô hình XGBoost
    print("Đang huấn luyện mô hình XGBoost...")
    xgb_results = build_xgboost_model(
        prepared_data, 
        max_depth=6, 
        learning_rate=0.05, 
        n_estimators=200, 
        early_stopping_rounds=20
    )
    print("Hoàn thành huấn luyện mô hình XGBoost")
    
    # Đánh giá mô hình
    actual = prepared_data['actual_test']
    lstm_predictions = lstm_results['predictions']
    xgb_predictions = xgb_results['predictions']
    
    evaluation_results = evaluate_models(actual[squence_length:], lstm_predictions, xgb_predictions[squence_length:])
    
    return  evaluation_results

In [71]:
file_path = "../data/processed/stock_data_final.csv"
list_stock = ['TMB', 'HGM', 'PVS', 'PVD', 'FPT', 'ITD', 'VIC', 'ACB', 'VCB','MBB']
eval_all = {}
preds_all = {}

for stock_code in list_stock:
    print(f"Đang xử lý cổ phiếu {stock_code}")
    evaluation_results, results_predict= run_stock_prediction(file_path, stock_code)
    eval_all[stock_code] = evaluation_results
    preds_all[stock_code] = results_predict


Đang xử lý cổ phiếu TMB
Bắt đầu dự đoán giá cổ phiếu TMB...
Đã đọc dữ liệu cho cổ phiếu TMB, số lượng mẫu: 1511
Hoàn thành chuẩn bị dữ liệu
Đang huấn luyện mô hình LSTM...
Epoch 1/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 0.0217 - val_loss: 0.0147
Epoch 2/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0037 - val_loss: 0.0082
Epoch 3/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0031 - val_loss: 0.0065
Epoch 4/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0033 - val_loss: 0.0103
Epoch 5/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0028 - val_loss: 0.0150
Epoch 6/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0023 - val_loss: 0.0049
Epoch 7/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.0029 - val_loss: 0.0060
Epoch 8/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0022 - val_loss: 0.0165
Epoch 9/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0027 - val_loss: 0.0105
Epoch 10/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0

In [73]:
lstm_data = {key: val['lstm'] for key, val in eval_all.items()}
xgboost_data = {key: val['xgboost'] for key, val in eval_all.items()}

# Chuyển dict thành DataFrame
df_lstm = pd.DataFrame.from_dict(lstm_data, orient='index')
df_xgboost = pd.DataFrame.from_dict(xgboost_data, orient='index')

# Hiển thị DataFrame
print("DataFrame cho LSTM:")
print(df_lstm)
print("\nDataFrame cho XGBoost:")
print(df_xgboost)

DataFrame cho LSTM:
          rmse       mae        r2         mse
TMB  11.372556  9.836015 -0.758238  129.335036
HGM  20.726708  9.844979  0.804910  429.596415
PVS   1.461444  1.156098  0.785508    2.135818
PVD   0.860414  0.659691  0.912588    0.740313
FPT   9.833605  8.444971  0.395307   96.699791
ITD   0.753949  0.495700  0.905604    0.568439
VIC   1.404730  0.991644  0.600114    1.973267
ACB   0.730364  0.536734  0.765298    0.533431
VCB   1.675215  1.295654  0.578424    2.806345
MBB   0.647680  0.498376  0.567509    0.419489

DataFrame cho XGBoost:
          rmse        mae         r2          mse
TMB  30.296131  29.062395 -11.477739   917.855581
HGM  50.908024  24.498028  -0.176918  2591.626920
PVS   2.783311   1.963365   0.222017     7.746822
PVD   0.783901   0.567435   0.927444     0.614500
FPT  28.296738  25.729726  -4.007051   800.705400
ITD   0.363893   0.258373   0.978010     0.132418
VIC   0.731893   0.503376   0.891446     0.535668
ACB   0.393383   0.279647   0.931912   

In [74]:
df_lstm.to_csv("../data/df_lstm.csv")
df_xgboost.to_csv("../data/df_xgboost.csv")

In [79]:
preds_all['TMB']['lstm']

{'actual': array([48. , 52.8, 55.7, 58.6, 59.8, 61.1, 61. , 61.1, 61.1, 61.1, 61.1,
        61.1, 61.1, 58.4, 58.4, 58.3, 57.4, 59.2, 59.8, 60.5, 60.5, 59.3,
        59.4, 58.6, 59.5, 59.5, 59.3, 58.4, 58.5, 58.8, 58.5, 59.4, 62. ,
        63.2, 62.6, 61.8, 61.6, 61.9, 61.7, 61.3, 63.1, 65. , 66.8, 68.9,
        71. , 71.9, 71.4, 75.2, 79. , 84.4, 85.1, 85.8, 86.6, 87.3, 86.4,
        85.4, 84.5, 85. , 85. , 84. , 82. , 81.8, 79.5, 77.2, 74.8, 72.5,
        72.5, 72.5, 71. , 70. , 71. , 72. , 73. , 74.5, 73.8, 73. , 71.2,
        71.3, 70. , 70.5, 70.9, 71.3, 70.1, 69.8, 68.8, 71.1, 70.5, 70. ,
        68.3, 69.5, 70.6, 71.8, 72.9, 74.1, 74.4, 74.8, 75.8, 76.7, 77. ,
        77.3, 77.3, 79.4, 79.1, 80.4, 81.2, 82. , 85. , 84. , 84. , 84. ,
        81.9, 83.1, 82. , 82. , 83.4, 82.5, 84.1, 86.2, 88.2, 89.8, 91.3,
        92.6, 93.9, 92.5, 92. , 91.5, 89.2, 89.9, 89.8, 85.9, 82. , 78.1,
        75.6, 71.4, 69.4, 68.6, 67.8, 70. , 69.9, 70.9, 72.5, 70.7, 70. ,
        68.5, 70.5, 71. , 70